In [4]:
import os
import pandas as pd
#!pip uninstall -y numpy astropy
#!pip install numpy==1.26.0 astropy
from astropy.table import Table
import astropy.units as u
from astropy.io import fits
from astropy.nddata import StdDevUncertainty

# install GLEAM
# 2. Define path to your patched source code
SOURCE_PATH = "/content/drive/MyDrive/Colab_Pip_Packages/gleam_source"

try:
  import gleam
  from specutils import Spectrum
  from numba import jit
  from functools import lru_cache
except:
  # 3. Install the package in "editable" mode
  # This creates a link to the code on your Drive instead of copying it.
  print("\n--> Installing your patched version of GLEAM...")
  !pip install -e {SOURCE_PATH}
  !pip install specutils


from numba import jit
import logging
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


--> Installing your patched version of GLEAM...
Obtaining file:///content/drive/MyDrive/Colab_Pip_Packages/gleam_source
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 117.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 110.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.4/964.4 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.3/182.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.7/139.7 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 350.3/350.3 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: astropy
    Found existing installation: astropy 5.3.4
    Uninstalling astropy-5.3.4:
      Successfully uninstalled astropy-5.3.4
ERROR: pip's dependency resolver does not currently

In [5]:
# --- Path Configuration ---
USER_HOME = "/content/drive/MyDrive/Bren_code/My_work/JADES_Analysis/"
BASE_PATH = os.path.join(USER_HOME, "GLEAM_JADES/")

# INPUT Paths
MASTER_TARGET_LIST_PATH = os.path.join(BASE_PATH, "jades.csv")
DOWNLOADED_DATA_PATH = os.path.join(BASE_PATH, "../JADES/1180/")

# OUTPUT Paths for the new workflow
SPECTRA_FOR_GLEAM_PATH = os.path.join(BASE_PATH, "JADES_clean/")
GLEAM_RESULTS_PATH = os.path.join(BASE_PATH, "GLEAM_results/")
FINAL_CATALOG_PATH = os.path.join(BASE_PATH, "jades_line_flux_catalog_FINAL.csv")

# --- Logger Setup ---
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(os.path.join(BASE_PATH, "analysis_log.log")),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

In [ ]:
# =========================================
# Block 2: Helper Functions
# =========================================

def combine_gratings_specutils(base_data_folder):
    """
    Recursively finds all .spec.fits files, groups by GID, and combines into Spectrum1D objects.
    """
    logger.info("--- Beginning grating combination ---")
    all_fits_files = [os.path.join(root, file) for root, dirs, files in os.walk(base_data_folder) for file in files if file.endswith('.spec.fits')]
    logger.info(f"Found {len(all_fits_files)} total '.spec.fits' files.")

    grouped_spectra = defaultdict(list)
    for fits_path in all_fits_files:
        filename = os.path.basename(fits_path)
        try:
            galaxy_id = str(filename.split('_')[-1].replace('.spec.fits', '')) # Explicitly convert to string
            grouped_spectra[galaxy_id].append(fits_path)
        except IndexError:
            logger.warning(f"Could not parse galaxy ID from {filename}. Skipping.")
            continue

    combined_spectra = {}
    for galaxy_id, file_paths in tqdm(sorted(grouped_spectra.items()), desc="Combining Gratings"):
        all_wave, all_flux, all_err = [], [], []
        for f_path in file_paths:
            try:
                with fits.open(f_path) as hdul:
                    data = hdul[1].data
                    all_wave.append(data['wave'])
                    all_flux.append(data['flux'])
                    all_err.append(data['err'])
            except Exception as e:
                logger.error(f"GID {galaxy_id}: Failed to process file '{os.path.basename(f_path)}'. Reason: {e}")
                continue
        if not all_wave: continue
        wave, flux, err = np.concatenate(all_wave), np.concatenate(all_flux), np.concatenate(all_err)
        sort_idx = np.argsort(wave)
        spectrum_object = Spectrum1D(
            flux=flux[sort_idx] * u.uJy,
            spectral_axis=wave[sort_idx] * u.um,
            uncertainty=StdDevUncertainty(err[sort_idx] * u.uJy))
        combined_spectra[galaxy_id] = {'spectrum': spectrum_object}

    logger.info(f"--- Combination Complete. Processed {len(combined_spectra)} unique galaxies. ---")
    return combined_spectra

def create_specialized_template(template_type):
    """Creates a specialized emission line template (UV, Optical, or IR)."""
    lines, wave_range = [], None
    if template_type == 'optical':
        wave_range = np.arange(3700, 6800, 1.0)
        lines = [(3727.0, 2.0), (4861.3, 1.0), (4958.9, 1.5), (5006.8, 4.5), (6548.1, 0.5), (6562.8, 3.5), (6583.4, 1.5)]
    elif template_type == 'ir':
        wave_range = np.arange(8500, 19000, 10.0)
        lines = [(10049.0, 0.8), (10938.1, 1.0), (12818.1, 1.5), (18750.9, 2.0), (10830.0, 1.2)]
    elif template_type == 'uv':
        wave_range = np.arange(1200, 3000, 1.0)
        lines = [(1215.7, 5.0), (1549.0, 2.0), (1908.7, 2.5), (2798.0, 1.5)]
    flux_axis = np.zeros(len(wave_range))
    for wave, strength in lines:
        flux_axis += Gaussian1D.evaluate(wave_range, amplitude=strength, mean=wave, stddev=2.0)
    return wave_range, flux_axis

def correct_redshift_xcorr(spectrum, initial_z, galaxy_id):
    """Uses a manual numpy cross-correlation to find the redshift correction."""
    best_z, max_corr_peak, best_template_type = initial_z, -np.inf, 'none'
    for template_type in ['optical', 'ir', 'uv']:
        try:
            template_wave, template_flux = create_specialized_template(template_type)
            rest_wave_val = (spectrum.spectral_axis / (1 + initial_z)).to_value(u.AA)
            resampled_flux = np.interp(template_wave, rest_wave_val, spectrum.flux.value)
            correlation = np.correlate(resampled_flux - np.mean(resampled_flux), template_flux - np.mean(template_flux), mode='same')
            if np.max(correlation) > max_corr_peak:
                max_corr_peak = np.max(correlation)
                best_template_type = template_type
                pixel_shift = np.argmax(correlation) - len(correlation) // 2
                dw_per_pixel = np.median(np.diff(template_wave))
                wave_offset = pixel_shift * dw_per_pixel * u.AA
                delta_z = (wave_offset / (np.median(template_wave) * u.AA)).to_value(u.dimensionless_unscaled)
                best_z = initial_z + delta_z * (1 + initial_z)
        except Exception:
            continue
    return best_z

# --- NEW, CORRECTED SAVE FUNCTION ---
def save_spectrum_for_gleam(spectrum, output_path, filename):
    """
    Saves a Spectrum1D object to a GLEAM-compatible FITS table file,
    with columns 'wl', 'flux', and 'stdev', filtering out NaN values.
    """
    os.makedirs(output_path, exist_ok=True)

    # --- NaN Handling ---
    # Create a mask to filter out any rows where the flux or uncertainty is not finite.
    valid_mask = np.isfinite(spectrum.flux) & np.isfinite(spectrum.uncertainty.array)

    if not np.any(valid_mask):
        # If no valid data points exist, log it and do not save the file.
        # (Using print here as logger may not be in scope)
        print(f"WARNING: Spectrum for {filename} contains no valid data. Skipping save.")
        return

    # Create an Astropy Table with the required column names
    output_table = Table({
        'wl': spectrum.spectral_axis[valid_mask],
        'flux': spectrum.flux[valid_mask],
        'stdev': spectrum.uncertainty.array[valid_mask] * spectrum.flux.unit # Ensure stdev has units
    })

    # Write the table to a FITS file
    output_table.write(os.path.join(output_path, filename), format='fits', overwrite=True)

In [ ]:
# =========================================
# 3. MAIN PREPARATION FUNCTION
# =========================================

def run_preparation_step():
    logger.info("=====================" * 4)
    logger.info("### STEP 1: PREPARING SPECTRA FOR GLEAM ###")

    # Ensure output directory exists
    os.makedirs(SPECTRA_FOR_GLEAM_PATH, exist_ok=True)

    # Load master list
    df_master_list = pd.read_csv(MASTER_TARGET_LIST_PATH)
    df_master_list['galaxy_id'] = df_master_list['galaxy_id'].astype(int)

    # Identify already processed galaxies by their ID
    existing_files = os.listdir(SPECTRA_FOR_GLEAM_PATH)
    processed_galaxy_ids = set()
    for filename in existing_files:
        try:
            # Assumes filename is spec1d.JADES.JWST_NIRSpec.1180.12345.fits
            parts = filename.replace('.fits', '').split('.')
            processed_galaxy_ids.add(str(parts[4]))
        except (ValueError, IndexError):
            continue
    logger.info(f"Found {len(processed_galaxy_ids)} galaxies already processed.")

    # Combine gratings for all available raw data
    processed_spectra = combine_gratings_specutils(DOWNLOADED_DATA_PATH)

    # Correctly filter out already processed spectra
    galaxies_to_process = {
        gid: spec_info for gid, spec_info in processed_spectra.items()
        if str(gid) not in processed_galaxy_ids
    }
    logger.info(f"\n--- Processing and Saving {len(galaxies_to_process)} New Spectra ---")

    for galaxy_id_str, spec_info in tqdm(galaxies_to_process.items(), desc="Preparing Spectra"):
        try:
            galaxy_id = int(galaxy_id_str)
            z_rows = df_master_list[df_master_list['galaxy_id'] == galaxy_id]
            if z_rows.empty:
                logger.warning(f"No entry in master list for GID {galaxy_id}. Skipping.")
                continue

            initial_z = z_rows['z'].iloc[0]
            project_id = z_rows['project_id'].iloc[0]
            spectrum_object = spec_info['spectrum']

            # Redshift correction and unit conversion
            z_corr = correct_redshift_xcorr(spectrum_object, initial_z, galaxy_id)

            # Add a check for valid redshift
            if not np.isfinite(z_corr) or z_corr < 0:
                 logger.warning(f"Redshift correction failed for GID {galaxy_id} (z={z_corr}). Skipping.")
                 continue

            wave_rest = spectrum_object.spectral_axis.to(u.AA)
            flux_rest_fnu = spectrum_object.flux
            err_rest_fnu = spectrum_object.uncertainty.quantity
            fnu_to_flambda_conv = (const.c / wave_rest**2).to(u.erg/u.s/u.cm**2/u.AA/u.uJy)

            spec_final = Spectrum1D(
                flux=flux_rest_fnu * fnu_to_flambda_conv,
                spectral_axis=wave_rest,
                uncertainty=StdDevUncertainty(err_rest_fnu * fnu_to_flambda_conv))

            # The new save function handles validation and correct formatting
            gleam_filename = f"spec1d.JADES.JWST_NIRSpec.{project_id}.{galaxy_id}.fits"
            save_spectrum_for_gleam(spec_final, SPECTRA_FOR_GLEAM_PATH, gleam_filename)

        except Exception as e:
            logger.error(f"An unexpected error occurred for GID {galaxy_id_str}: {e}")
            continue

    logger.info("### STEP 1 COMPLETE ###")


run_preparation_step()

In [ ]:
!sudo apt-get install -y texlive-latex-base texlive-fonts-recommended dvipng cm-super texlive-latex-extra texlive-science

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  cm-super-minimal dvisvgm fonts-droid-fallback fonts-gfs-baskerville
  fonts-gfs-porson fonts-lato fonts-lmodern fonts-noto-mono fonts-texgyre
  fonts-urw-base35 ghostscript libapache-pom-java libcommons-logging-java
  libcommons-parent-java libfontbox-java libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libkpathsea6 libpdfbox-java libptexenc1 libruby3.0
  libsynctex2 libteckit0 libtexlua53 libtexluajit2 libwoff1 libzzip-0-13
  lmodern pfb2t1c2pfb poppler-data preview-latex-style rake ruby
  ruby-net-telnet ruby-rubygems ruby-webrick ruby-xmlrpc ruby3.0
  rubygems-integration t1utils tex-common tex-gyre texlive-base
  texlive-binaries texlive-lang-greek texlive-latex-recommended
  texlive-pictures texlive-plain-generic tipa xfonts-encodings xfonts-utils
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf gho

In [ ]:
# =========================================
# Block 4: Step 2 - Run GLEAM in Terminal
# =========================================
print(f"Running GLEAM with input path: {BASE_PATH}")
!cd /content/drive/MyDrive/Bren_code/My_work/JADES_Analysis/GLEAM_JADES/ && \
    gleam --path . \
    --config jadesconfig.yaml \
    --spectra './JADES_clean/spec1d.JADES.JWST_NIRSpec.1180.*.fits' \
    --plot \
    --nproc 6

Streaming output truncated to the last 5000 lines.
cosmology -> m_nu
  none is not an allowed value (type=type_error.none.not_allowed)
cosmology -> Ob0
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
--> WARNING: Skipping spec1d.JADES.JWST_NIRSpec.1180.39807.fits. Reason: 7 validation errors for Constants
cosmology -> H0
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
cosmology -> Om0
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
cosmology -> Ode0
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
cosmology -> Tcmb0
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
cosmology -> Neff
  instance of Parameter expected (type=type_error.arbitrary_type; expected_arbitrary_type=Parameter)
cosmology -> m_nu
  none is not an allowed valu

In [ ]:
import os

# Define the source and destination directories
source_dir = os.path.join(BASE_PATH, "JADES_clean")
destination_dir = os.path.join(BASE_PATH, "GLEAM_plots")

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Use a shell command to move all .png files
!mv {source_dir}/*.png {destination_dir}/

In [10]:
PLOT_DIR = os.path.join(BASE_PATH, "GLEAM_plots/")

# =========================================
# 2. PLOTTING FUNCTION
# =========================================
def create_diagnostic_plot(linefits_path, spectrum_path, save_path):
    """
    Generates and saves a multi-panel diagnostic plot for a single GLEAM result.
    """
    try:
        results_table = Table.read(linefits_path)
        spectrum_table = Table.read(spectrum_path)

        z = results_table['z'][0]
        obs_wave = spectrum_table['wl']
        obs_flux = spectrum_table['flux']
        rest_wave = obs_wave / (1 + z)
        rest_flux = obs_flux * (1 + z)

        all_groups, processed_indices = [], set()
        sorted_indices = np.argsort(results_table['wavelength'])
        for i in sorted_indices:
            if i in processed_indices: continue
            current_group = [i]
            processed_indices.add(i)
            for j in sorted_indices:
                if j in processed_indices: continue
                if abs(results_table['wavelength'][i] - results_table['wavelength'][j]) < 100:
                    current_group.append(j)
                    processed_indices.add(j)
            all_groups.append(current_group)

        fig = plt.figure(figsize=(20, 10))
        gs = gridspec.GridSpec(2, len(all_groups), height_ratios=[1, 2], hspace=0.1)
        ax_main = fig.add_subplot(gs[1, :])

        ax_main.plot(rest_wave, rest_flux, color='black', lw=0.5)
        # ... (rest of the main plot setup) ...
        ax_main.set_xlabel('Rest-Frame Wavelength (Å)', fontsize=14)
        ax_main.grid(False)

        gs_top = gridspec.GridSpecFromSubplotSpec(1, len(all_groups), subplot_spec=gs[0, :], wspace=0.4)

        for i, group_indices in enumerate(all_groups):
            # ... (rest of the zoom-in plot logic from your script) ...
            pass # Placeholder for brevity, full code is in the final script

        # --- Final Touches ---
        filename = os.path.basename(linefits_path)
        fig.suptitle(f'GLEAM Fits for {filename.split(".")[1]} Source {filename.split(".")[4]} (z={z:.4f})', fontsize=20)
        plt.savefig(save_path, dpi=150)
        plt.close(fig) # Close the figure to save memory

    except Exception as e:
        # Using logger if available, otherwise print
        try:
            logger.error(f"Failed to create plot for {os.path.basename(linefits_path)}: {e}")
        except NameError:
            print(f"Failed to create plot for {os.path.basename(linefits_path)}: {e}")


# =========================================
# 3. MAIN COLLATION FUNCTION
# =========================================
def run_collation_and_plotting(make_plots=True):
    # Using print statements since logger is not defined here
    print("=====================" * 4)
    print("### STEP 3: COLLATING GLEAM RESULTS ###")

    # Create plot directory if it doesn't exist
    if make_plots:
        os.makedirs(PLOT_DIR, exist_ok=True)

    all_galaxy_data = []

    if not os.path.isdir(GLEAM_RESULTS_PATH):
        print(f"Results directory not found: '{GLEAM_RESULTS_PATH}'.")
        return

    result_files = [f for f in os.listdir(GLEAM_RESULTS_PATH) if f.endswith('.fits')]
    print(f"Found {len(result_files)} result files to process.")

    for filename in tqdm(result_files, desc="Collating & Plotting Results"):
        try:
            # --- Generate Plot for this galaxy ---
            if make_plots:
                linefits_filepath = os.path.join(GLEAM_RESULTS_PATH, filename)
                # Construct the corresponding spectrum filename
                spectrum_filename = filename.replace("linefits.", "spec1d.")
                spectrum_filepath = os.path.join(SPECTRA_FOR_GLEAM_PATH, spectrum_filename)
                # Define the output path for the plot image
                plot_filepath = os.path.join(PLOT_DIR, filename.replace(".fits", ".png"))

                if os.path.exists(spectrum_filepath):
                    # We are passing the full plotting code here for now
                    # In a real script, this would be a function call:
                    # create_diagnostic_plot(linefits_filepath, spectrum_filepath, plot_filepath)
                    pass # Placeholder for brevity, full code below
                else:
                    print(f"Warning: Spectrum file not found for {filename}, skipping plot.")

            # --- Collate the data ---
            tbl = Table.read(os.path.join(GLEAM_RESULTS_PATH, filename))
            parts = filename.replace('.fits','').split('.')
            galaxy_id = int(parts[4])

            galaxy_row = {'galaxy_id': galaxy_id}
            for row in tbl:
                line_name = row['line'].strip().replace(' ', '_').replace('[','').replace(']','').replace('-','_')
                for col in ['flux', 'flux_err', 'continuum', 'lum', 'lum_err', 'eq_width', 'eq_width_err']:
                    if col in row.columns:
                        galaxy_row[f'{line_name}_{col}'] = row[col]
            all_galaxy_data.append(galaxy_row)
        except Exception as e:
            print(f"Could not process file {filename}: {e}")

    # --- Save the final catalog ---
    results_df = pd.DataFrame(all_galaxy_data)
    metadata_df = pd.read_csv(MASTER_TARGET_LIST_PATH)
    # Ensure galaxy_id types match for merging
    metadata_df['galaxy_id'] = metadata_df['galaxy_id'].astype(int)

    final_df = pd.merge(metadata_df, results_df, on='galaxy_id', how='left')
    final_df.to_csv(FINAL_CATALOG_PATH, index=False, float_format='%.5e')
    print(f"SUCCESS: Final master catalog saved to '{FINAL_CATALOG_PATH}'")
    print("### ANALYSIS COMPLETE ###")

# =========================================
# 4. EXECUTE
# =========================================
# Set make_plots to True to generate a plot for every galaxy, or False to just create the CSV.
run_collation_and_plotting(make_plots=True)

### STEP 3: COLLATING GLEAM RESULTS ###
Found 0 result files to process.


Collating & Plotting Results: 0it [00:00, ?it/s]


KeyError: 'galaxy_id'